# Update units of attribute files
The values were calculated correctly, but ended up with incorrect units in the attribute files. Here we fix these.

- attributes-lumped:
  - rainfall-runoff ratio units
  - PET
  - P
- attributes-distributed:
  - PET
  - P

In [1]:
import warnings
import pandas as pd
from pathlib import Path

In [2]:
# Data location
cs_main_folder = Path("/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload")

In [3]:
# Destination location
cs_update_folder = Path("/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload-updates")

In [4]:
# Specify the folder structure
att_path_part1 = "attributes"
att_path_parts2 = ["headwater", "macro-scale", "meso-scale"]

## Functions

In [6]:
def replace_attribute_units(df):

    '''Replaces units in an existing CAMELS-SPAT attribute file'''

    # Define what to update (attribute, new unit)
    update_this = {
        # Climate attributes
        'PR0_mean': 'mm year-1',
        'PR0_std': 'mm year-1',
        'pet1_mean': 'mm year-1',
        'pet1_std': 'mm year-1',
        'prec_mean': 'mm year-1',
        'prec_std': 'mm year-1',
        'pet2_mean': 'mm year-1',
        'pet2_std': 'mm year-1',
                    
        # Streamflow attributes 
        'runoff_ratio_mean': '-',
        'runoff_ratio_std': '-',
   }

    update_this_month = {
        # Climate attributes
        'pet1_mean_month_{}': 'mm month-1',
        'pet1_std_month_{}': 'mm month-1',
        'PR0_mean_month_{}': 'mm month-1',
        'PR0_std_month_{}': 'mm month-1',
        'pet2_mean_month_{}': 'mm month-1',
        'pet2_std_month_{}': 'mm month-1',
        'prec_mean_month_{}': 'mm month-1',
        'prec_std_month_{}': 'mm month-1',
    }
    
    # Update the fixed attributes
    for attribute, unit in update_this.items():        
        df.loc[df["Attribute"] == attribute, "Unit"] = unit

    # Update the monthly attributes
    for attribute, unit in update_this_month.items():
        for month in range(1,13):
            df.loc[df["Attribute"] == attribute.format(f"{month:02}"), "Unit"] = unit

    return df

## Lumped attributes

In [6]:
# Define location
att_file = "attributes-lumped.csv"
att_path = cs_main_folder / att_path_part1 / att_file

In [11]:
# Open file
att = pd.read_csv(att_path, low_memory=False)

In [31]:
# Replace units
att_new = replace_attribute_units(att)

In [38]:
# Save to file
des_folder = cs_update_folder / att_path_part1 
des_folder.mkdir(exist_ok=True, parents=True)
att_new.to_csv(des_folder / att_file, index=False)

## Distributed attributes

In [9]:
for att_path_part2 in att_path_parts2:

    # 1. Find the files in this folder
    att_middle = f"{att_path_part1}/{att_path_part2}"
    basin_files = list((cs_main_folder / att_middle).glob('*.csv'))

    # 2. Loop over the files
    for basin_file in basin_files:

        # 2.1. Extract the filename
        att_file = basin_file.name

        # 2.2. Load the file
        att = pd.read_csv(basin_file, low_memory=False)

        # 2.3. Replace units
        att_new = replace_attribute_units(att)

        # 2.4. Save to file
        des_folder = cs_update_folder / att_middle 
        des_folder.mkdir(exist_ok=True, parents=True)
        att_new.to_csv(des_folder / att_file, index=False)